### FSPS implementation in SKIRT: stab file
This script will take FSPS (https://github.com/cconroy20/fsps) data output and make an interpolation to increase the resolution ($R= \frac{ \Delta \lambda}{\lambda}$) as stated in the document by Baes, Camps, Kapoor & Rivas-Sánchez 2022.

J. Fritz & M Rivas-Sánchez:

"When we set up a library, one sensible option could be to assume a fixed spectral resolution for the lines,
meaning that we choose a value R, and determine the width $\Delta \lambda_{k}$ of each line according to

$\Delta \lambda_{k} = \frac{2\lambda_{k}}{R}$ or $R = \frac{2\lambda_{k}}{\Delta \lambda_{k}} $."

$FWHM = 2 \sqrt{2ln(2)} \sigma$

"If we know that the area under the Gaussian curve is ∼ 99.993% when we reach 4$\sigma$ from the center, we can safely truncate the Gaussian profile at 4$\sigma$. For a given intrinsic wavelength resolution R, we can then determine the standard deviation of the k’th line $\sigma_{k}$ as:

$\sigma_{k} = \frac{\Delta \lambda_{k}}{8} = \frac{\lambda_{k}}{4R}$."


In [1]:
#Importing libraries
import fsps
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
import pts.storedtable.io as stab
from astropy.visualization import astropy_mpl_style
import astropy.io.ascii as ascii
from __future__ import print_function

In [14]:
#Technical specifications
nint = 37 #number of new points per emission lines
fwhm2sig = 2.*np.sqrt(np.log(2)) #relationship between fwhm and sigma
#fsps resolution (fwhm)
fwhm_hires, fwhm_lores = 2.3, 30.
#solving for sigma  
sig_hires, sig_lores = fwhm_hires/fwhm2sig, fwhm_lores/fwhm2sig
#delta resolution
Delta_hires, Delta_lores = 4.*sig_hires, 4.*sig_lores
#FSPS [3600,7400] Ang resolution FWHM=2.3 Ang
l1, l2 = 3600., 7400.

#Reading files
llines = ascii.read('/Users/.../fsps_lumlines.txt')
nlines = len(llines)
lc = np.sort(llines) #increasing wavelength lines 

spectrum = ascii.read('/Users/.../data_fsps.txt')
nwav = len(spectrum) #number of wavelengths
wavel = spectrum[0][:] #wavelength in Ang

In [9]:
#Creating new array for interpolated wavelengths
int_lin_array = []
new_array = []
llim = 0

for _i in range(nlines):
    if lc[_i][0] >= l1 and lc[_i][0] <= l2: #Miles resolution
        delta = Delta_hires
    else:
        delta = Delta_lores
    li = lc[_i][0] - delta #lower
    lu = lc[_i][0] + delta #upper
    dum_wave = wavel[np.where(wavel < li)]
    wave = wavel[np.where(dum_wave > llim)]
    llim = lu
    new_array = np.append(new_array, wave) #
    dl = (lu-li)/float(nint-1)
    dum_arr = [(li + dl*n) for n in range(nint)] #new wavelength points
    new_array = np.append(new_array, dum_arr)

new_wavel = np.sort(new_array)

In [6]:
"""
This fragment was taken from:
https://skirt.ugent.be/pts9/namespacepts_1_1storedtable_1_1do_1_1fsps__with__lines__to__stored__table.html

# *****************************************************************
# **       PTS -- Python Toolkit for working with SKIRT          **
# **       © Astronomical Observatory, Ghent University          **
# *****************************************************************
"""
# define the script body inside a function to avoid execution at the top level
def do() -> "Create FSPS-generated SED family including emission lines as stored table":
    # the file path for the .stab output file (here placed in the current directory)
    stabPath = "FSPS-emissionlines.stab"

    # the metallicity and ionization parameter grids
    Zsun = 0.0142 #Padova = 0.0190
    logZv = [-2.5, -2, -1.5, -1, -0.6, -0.4, -0.3, -0.2, -0.1, 0, 0.1, 0.2, 0.5]
    logUv = [-4, -3.5, -3, -2.5, -2, -1.5, -1]

    # initialize the ssp object (it is advised to do this only once)
    print("Initializing FSPS...", flush=True)
    sp = fsps.StellarPopulation(compute_vega_mags = False,
                                zcontinuous=1,               # Type of interpolation for metallicity
                                sfh=0,                       # SSPs
                                imf_type=2,                  # Kroupa IMF
                                add_dust_emission=False,     # Turn off dust emission
                                add_neb_emission=True,       # Turn on nebular emission
                                add_neb_continuum=True,
                                nebemlineinspec=True,        # Removes lines from spectra
                                cloudy_dust=False,           # Exclude dust from nebular emission
                                logzsol=-2,                  # Stellar metallicity
                                gas_logz=-2,                 # Gas metallicity (must be the same as stellar)
                                gas_logu=-4)                 # Ionization parameter
    
    print (sp.libraries)

    # perform a dummy calculation to get the wavelength and age grids
    # (if we don't specify the age, a data cube with SSPs for a range of ages is created)
    print("Determining grids...", flush=True)
    wv, _ = sp.get_spectrum(peraa=True)     # wavelengths in Angstrom
    logtv = sp.log_age
    tv = 10.**logtv                         # ages in year
    Zv = Zsun * 10.**np.array(logZv)        # metallicities from log(Z/Zsun)
    Uv = 10.**np.array(logUv)

    # allocate hypercube for the luminosities
    L = np.zeros((len(new_wavel), len(Zv), len(tv), len(Uv)))

    # perform calculations for all metallicities and ionization parameters
    for iZ, logZ in enumerate(logZv):
        for iU, logU in enumerate(logUv):
            print("Calculating spectra for logZ={} and logU={}...".format(logZ, logU), flush=True)

            # modify parameters in the sp object
            sp.params['logzsol'] = logZ
            sp.params['gas_logz'] = logZ
            sp.params['gas_logu'] = logU

            # get specific luminosities in Lsun/Angstrom and put them into the hypercube
            _, final_spectra = sp.get_spectrum(peraa=True)
            
            Lv = np.zeros((final_spectra.shape[0],new_wavel.shape[0]))

            for i in range(Lv.shape[0]):
                Lv[i,:] = np.interp(new_wavel, _, final_spectra[i,:])
            
            L[:, iZ, :, iU] = Lv.T

    # convert from input units Lsun/Angstrom to output units W/m
    L *= 3.848e26 * 1e10
    wv *= 1e-10

    # write the stored table
    print("Writing stored table file {}...".format(stabPath), flush=True)
    stab.writeStoredTable(stabPath,
            ['lambda','Z','t','U'], ['m','1','yr','1'], ['log','log','log','log'], [new_wavel,Zv,tv,Uv],
            ['Llambda'], ['W/m'], ['log'], [L])
    print()
    print("Done.", flush=True)


# remove the "#" in front of the next line to perform the script body
do()


Initializing FSPS...
(b'mist', b'miles', b'DL07  ')
Determining grids...
Calculating spectra for logZ=-2.5 and logU=-4...
Calculating spectra for logZ=-2.5 and logU=-3.5...
Calculating spectra for logZ=-2.5 and logU=-3...
Calculating spectra for logZ=-2.5 and logU=-2.5...
Calculating spectra for logZ=-2.5 and logU=-2...
Calculating spectra for logZ=-2.5 and logU=-1.5...
Calculating spectra for logZ=-2.5 and logU=-1...
Calculating spectra for logZ=-2 and logU=-4...
Calculating spectra for logZ=-2 and logU=-3.5...
Calculating spectra for logZ=-2 and logU=-3...
Calculating spectra for logZ=-2 and logU=-2.5...
Calculating spectra for logZ=-2 and logU=-2...
Calculating spectra for logZ=-2 and logU=-1.5...
Calculating spectra for logZ=-2 and logU=-1...
Calculating spectra for logZ=-1.5 and logU=-4...
Calculating spectra for logZ=-1.5 and logU=-3.5...
Calculating spectra for logZ=-1.5 and logU=-3...
Calculating spectra for logZ=-1.5 and logU=-2.5...
Calculating spectra for logZ=-1.5 and logU=